## Experiment 1 (failed)

In [1]:
from gensim.models.word2vec import Word2Vec
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [2]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        if len(word2vec)>0:
            self.dim=len(word2vec[next(iter(glove_small))])
        else:
            self.dim=0
            
    def fit(self, X, y):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

    
# and a tf-idf version of the same
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        if len(word2vec)>0:
            self.dim=len(word2vec[next(iter(glove_small))])
        else:
            self.dim=0
        
    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf, 
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])
    
        return self
    
    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [50]:
import numpy as np
with open(rose['text'], "rb") as lines:
    wvec = {line.split()[0].decode(encoding): np.array(line.split()[1:],dtype=np.float32)
               for line in lines}

TypeError: expected str, bytes or os.PathLike object, not Series

In [3]:
rose = pd.read_excel('rose.xlsx')

In [18]:
text = rose['text']

In [19]:
count_vect = CountVectorizer()
text_counts = count_vect.fit_transform(text)
text_counts.shape

(15, 132)

In [37]:
w2v.items()

dict_items([('e', array([-0.02027582, -0.01239222,  0.01349756,  0.04614235, -0.03881368,
       -0.01403142, -0.05428512,  0.03362289,  0.0018558 , -0.00130165],
      dtype=float32)), ('n', array([-0.03825945, -0.02811386, -0.04106112,  0.03083437,  0.01579047,
       -0.01286134, -0.01877186, -0.01159137, -0.00805879,  0.04646649],
      dtype=float32)), ('t', array([ 0.02675523,  0.00614828,  0.0369582 ,  0.00727019,  0.00831095,
        0.04119881, -0.04494306, -0.02176107,  0.00568487, -0.0135679 ],
      dtype=float32)), ('s', array([-0.00458   , -0.00543844, -0.03407583, -0.00620107,  0.04362049,
        0.0066507 , -0.03984213,  0.0352293 ,  0.00365619,  0.00470684],
      dtype=float32)), ('o', array([ 0.00030056, -0.02137967,  0.00631244,  0.0297047 , -0.03652196,
        0.00408429, -0.03630996, -0.04031385, -0.03121908, -0.03324112],
      dtype=float32)), ('i', array([ 0.04612219, -0.04068083,  0.00113151, -0.03134857,  0.0058077 ,
        0.04784923, -0.01483676,  0.0479

In [44]:
# train word2vec on all the texts - both training and test set
# we're not using test labels, just texts so this is fine
model = Word2Vec(count_vect.get_feature_names(), size=100, window=5, min_count=5, workers=2)
w2v = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}

C:\Users\sureshsu\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "
C:\Users\sureshsu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  after removing the cwd from sys.path.


In [48]:
m = MeanEmbeddingVectorizer(w2v)

NameError: name 'glove_small' is not defined

In [46]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier

etree_w2v = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)),
    ("extra trees", ExtraTreesClassifier(n_estimators=200))])

AttributeError: 'dict' object has no attribute 'itervalues'

## Experiment 2 
http://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code/#.XO7n94hKg2x

In [51]:
rose

,text,target
0,this is a pink rose,0
1,the pink rose grows in the mud,0
2,the pink rose is not black,1
3,everybody like the pink rose,1
4,the blue rose is not black,1
5,blue roses are rare,1
6,Unlimited talk and text to US and Canada. Our ...,0
7,"Text messaging, or texting, is the act of comp...",1
8,Textfree is the free texting and free calling ...,0
9,Send & receive text messages in Messages. You ...,2


In [53]:
rose['text'].to_csv('rosetext.csv', index=False)

C:\Users\sureshsu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [57]:
with open('rosetext.csv', 'rb') as f:
    for i,line in enumerate (f):
        print(line)
#        break

b'this is a pink rose\r\n'
b'the pink rose grows in the mud\r\n'
b'the pink rose is not black\r\n'
b'everybody like the pink rose\r\n'
b'the blue rose is not black\r\n'
b'blue roses are rare\r\n'
b"Unlimited talk and\xc2\xa0text\xc2\xa0to US and Canada. Our phones come with a 30-day money-back guarantee and 1 year warranty. If you aren't satisfied just send your\xc2\xa0.\r\n"
b'"Text\xc2\xa0messaging, or\xc2\xa0texting, is the act of composing and sending electronic messages, typically consisting of alphabetic and numeric characters, between two or more users of mobile devices, desktops/laptops, or other type of compatible computer."\r\n'
b'"Textfree is the free texting and free calling app that gives you free\xc2\xa0text\xc2\xa0plus a real US phone number so you can\xc2\xa0text\xc2\xa0anyone, even if they don\'t have the app."\r\n'
b'Send & receive\xc2\xa0text\xc2\xa0messages in Messages. You can send and receive\xc2\xa0text\xc2\xa0messages with friends and contacts on Messages.\r\n'


In [63]:
def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    with open(input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("read {0} reviews".format (i))
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess (line)



In [61]:
import logging

In [64]:
# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input ('rosetext.csv'))
logging.info ("Done reading data file")

In [65]:
documents

[['this', 'is', 'pink', 'rose'],
 ['the', 'pink', 'rose', 'grows', 'in', 'the', 'mud'],
 ['the', 'pink', 'rose', 'is', 'not', 'black'],
 ['everybody', 'like', 'the', 'pink', 'rose'],
 ['the', 'blue', 'rose', 'is', 'not', 'black'],
 ['blue', 'roses', 'are', 'rare'],
 ['unlimited',
  'talk',
  'and',
  'text',
  'to',
  'us',
  'and',
  'canada',
  'our',
  'phones',
  'come',
  'with',
  'day',
  'money',
  'back',
  'guarantee',
  'and',
  'year',
  'warranty',
  'if',
  'you',
  'aren',
  'satisfied',
  'just',
  'send',
  'your'],
 ['text',
  'messaging',
  'or',
  'texting',
  'is',
  'the',
  'act',
  'of',
  'composing',
  'and',
  'sending',
  'electronic',
  'messages',
  'typically',
  'consisting',
  'of',
  'alphabetic',
  'and',
  'numeric',
  'characters',
  'between',
  'two',
  'or',
  'more',
  'users',
  'of',
  'mobile',
  'devices',
  'desktops',
  'laptops',
  'or',
  'other',
  'type',
  'of',
  'compatible',
  'computer'],
 ['textfree',
  'is',
  'the',
  'free',
 

In [66]:
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

(501, 2690)

In [68]:
model.wv.most_similar(positive='text')

[('real', 0.17798131704330444),
 ('login', 0.1543339490890503),
 ('calling', 0.1363137662410736),
 ('us', 0.1281176060438156),
 ('of', 0.12339971959590912),
 ('send', 0.11913784593343735),
 ('through', 0.1178809329867363),
 ('canada', 0.11634750664234161),
 ('business', 0.09261417388916016),
 ('free', 0.09156455099582672)]

In [78]:
len(model.wv.syn0)

C:\Users\sureshsu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


49

## Experiment 3 
https://www.kaggle.com/arunava21/word2vec-and-random-forest-classification

In [1]:
from gensim.models import Word2Vec, word2vec
import gensim
import logging
import matplotlib.pyplot as plt
import nltk
import numpy as np
from nltk.corpus import stopwords
import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline
import sqlite3
import re
from tqdm import tqdm
from sklearn.metrics import accuracy_score

C:\Users\sureshsu\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#rose = pd.read_excel('rose.xlsx')
rose = pd.read_excel('incident V2 - Enriched.xlsx')
rose.shape

(34564, 45)

In [3]:
rose = rose[rose['state'].isin(['Closed', 'Closed (CR Implemented)','Closed (Purchase Required)', 'Resolved'])].copy()
rose.shape

(30874, 45)

In [4]:
rose = rose[['description', 'ag']].copy()
rose.columns = ['text', 'target']
rose.shape

(30874, 2)

In [5]:
rose['text'] = rose['text'].astype(str)

In [6]:
train_size = int(len(rose) * 0.8)
train_reviews = rose.iloc[:train_size,:]
test_reviews = rose.iloc[train_size:,:]

In [7]:
train_reviews.shape, test_reviews.shape

((24699, 2), (6175, 2))

In [8]:
def review_to_wordlist(review, remove_stopwords=False):
    """
    Convert a review to a list of words. Removal of stop words is optional.
    """
    # remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review)
    
    # convert to lower case and split at whitespace
    words = review_text.lower().split()
    
    # remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    return words

In [9]:
def review_to_sentences(review, tokenizer, remove_stopwords=False):
    """
    Split review into list of sentences where each sentence is a list of words.
    Removal of stop words is optional.
    """
    # use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())

    # each sentence is furthermore split into words
    sentences = []    
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
            
    return sentences

In [10]:
# Load the punkt tokenizer used for splitting reviews into sentences
import nltk
nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sureshsu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
import re
train_sentences = []  # Initialize an empty list of sentences
for review in train_reviews['text']:
    train_sentences += review_to_sentences(review, tokenizer)

In [12]:
model = gensim.models.Word2Vec(train_sentences, size=128, window=2, min_count=2, workers=4, iter=10)

In [13]:
model.train(train_sentences,total_examples=len(train_sentences),epochs=10)

(70108282, 89512550)

In [14]:
model.wv.syn0.shape

C:\Users\sureshsu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


(46837, 128)

In [15]:
def make_feature_vec(words, model, num_features):
    """
    Average the word vectors for a set of words
    """
    feature_vec = np.zeros((num_features,),dtype="float32")  # pre-initialize (for speed)
    nwords = 0.
    index2word_set = set(model.wv.index2word)  # words known to the model

    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            feature_vec = np.add(feature_vec,model[word])
    
    feature_vec = np.divide(feature_vec, nwords)
    return feature_vec


def get_avg_feature_vecs(reviews, model, num_features):
    """
    Calculate average feature vectors for all reviews
    """
    counter = 0
    review_feature_vecs = np.zeros((len(reviews),num_features), dtype='float32')  # pre-initialize (for speed)
    
    for review in reviews:
        review_feature_vecs[counter] = make_feature_vec(review, model, num_features)
        counter = counter + 1
    return review_feature_vecs

In [112]:
counter = 0

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sureshsu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
# calculate average feature vectors for training and test sets
num_features=128
clean_train_reviews = []
for review in train_reviews['text']:
    clean_train_reviews.append(review_to_wordlist(review, remove_stopwords=True))
trainDataVecs = get_avg_feature_vecs(clean_train_reviews, model, num_features)

C:\Users\sureshsu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
C:\Users\sureshsu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


In [18]:
clean_test_reviews = []
for review in test_reviews['text']:
    clean_test_reviews.append(review_to_wordlist(review, remove_stopwords=True))
testDataVecs = get_avg_feature_vecs(clean_test_reviews, model, num_features)

C:\Users\sureshsu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
C:\Users\sureshsu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


In [19]:
trainDataVecs.shape

(24699, 128)

In [20]:
# remove instances in train set that could not be represented as feature vectors
nan_indices = list({x for x,y in np.argwhere(np.isnan(trainDataVecs))})
if len(nan_indices) > 0:
    print('Removing {:d} instances from train set.'.format(len(nan_indices)))
    trainDataVecs = np.delete(trainDataVecs, nan_indices, axis=0)
    train_reviews.drop(train_reviews.iloc[nan_indices, :].index, axis=0, inplace=True)
    assert trainDataVecs.shape[0] == len(train_reviews)

Removing 78 instances from train set.


C:\Users\sureshsu\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [21]:
# Fit a random forest to the training data, using 100 trees
forest = RandomForestClassifier(n_estimators = 128, n_jobs=-1)

print("Fitting a random forest to labeled training data...")
forest = forest.fit(trainDataVecs, train_reviews['target'])

Fitting a random forest to labeled training data...


In [22]:
# remove instances in test set that could not be represented as feature vectors
nan_indices = list({x for x,y in np.argwhere(np.isnan(testDataVecs))})
if len(nan_indices) > 0:
    print('Removing {:d} instances from test set.'.format(len(nan_indices)))
    testDataVecs = np.delete(testDataVecs, nan_indices, axis=0)
    test_reviews.drop(test_reviews.iloc[nan_indices, :].index, axis=0, inplace=True)
    assert testDataVecs.shape[0] == len(test_reviews)

Removing 20 instances from test set.


C:\Users\sureshsu\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [23]:
print("Predicting labels for test data..")
result = forest.predict(testDataVecs)

Predicting labels for test data..


In [24]:
accuracy_score(test_reviews['target'], result)

0.35743298131600326

In [49]:
model.wv.most_similar('helpdesk', topn=100)

[('help', 0.47340360283851624),
 ('consult', 0.45881205797195435),
 ('desk', 0.44597360491752625),
 ('bsas', 0.4140302538871765),
 ('dbas', 0.4030960202217102),
 ('dba', 0.40267786383628845),
 ('colleagues', 0.3985521197319031),
 ('procurement', 0.3808389902114868),
 ('billruncomm', 0.37541571259498596),
 ('esd', 0.3704761564731598),
 ('wnexmb', 0.3647332191467285),
 ('comunidade', 0.35747382044792175),
 ('servicenow', 0.3564300537109375),
 ('log', 0.3546685576438904),
 ('paysonnel', 0.3480268120765686),
 ('friends', 0.34560897946357727),
 ('itsoc', 0.34555569291114807),
 ('contact', 0.34363847970962524),
 ('iprocurement', 0.3427566885948181),
 ('assistance', 0.3412788510322571),
 ('bsa', 0.33042746782302856),
 ('hyperian', 0.3303166627883911),
 ('ticket', 0.3291030824184418),
 ('snow', 0.3228023052215576),
 ('collegues', 0.32042068243026733),
 ('mobility', 0.3191124200820923),
 ('hookup', 0.31502488255500793),
 ('sir', 0.3139888644218445),
 ('mnda', 0.31268274784088135),
 ('purchasing

In [44]:
model.wv.most_similar(['finance'], topn=10)

[('partnercentral', 0.5624165534973145),
 ('jpebsitsupport', 0.4998823404312134),
 ('esdecoleadsupport', 0.4862039387226105),
 ('ibxops', 0.4847724735736847),
 ('operations', 0.4780731201171875),
 ('apacordersupport', 0.4762503504753113),
 ('financial', 0.47356581687927246),
 ('legalemeasupport', 0.4623432755470276),
 ('capmgmt', 0.45611822605133057),
 ('apac', 0.44485199451446533)]

In [33]:
model.wv.most_similar('legal')

[('qrs', 0.5401618480682373),
 ('country', 0.5231621861457825),
 ('trs', 0.4748820662498474),
 ('german', 0.4576883316040039),
 ('le', 0.4484444260597229),
 ('rollup', 0.43891143798828125),
 ('employees', 0.4313257336616516),
 ('acquisition', 0.43077361583709717),
 ('whose', 0.425498366355896),
 ('account', 0.419788122177124)]

In [34]:
model.wv.most_similar('vertex')

[('iprocurement', 0.47603046894073486),
 ('sit', 0.47257912158966064),
 ('production', 0.4719308018684387),
 ('billtest', 0.4475070834159851),
 ('revpro', 0.4454643726348877),
 ('sv', 0.42614835500717163),
 ('bi', 0.42557933926582336),
 ('consistency', 0.42504629492759705),
 ('clone', 0.4220660924911499),
 ('ebs', 0.4150271415710449)]